# Importing Libraries

In [539]:
import pandas as pd
import numpy as np

# Loading the csv file

In [548]:
okc = pd.read_csv('shots_data.csv')

In [549]:
okc

,team,x,y,fgmade
0,Team A,-23.1,3.5,0
1,Team A,0.0,25.1,1
2,Team A,0.5,1.0,1
3,Team A,-5.6,5.2,0
4,Team A,4.0,14.5,1
...,...,...,...,...
555,Team B,7.8,24.7,0
556,Team B,10.1,1.4,1
557,Team B,-17.8,-0.1,0
558,Team B,22.6,2.9,0


# Determine radius of the arc with x,y co-ordinates

In [550]:
okc['rad_crit'] = np.sqrt(okc['x']**2 + okc['y']**2)

In [551]:
okc

,team,x,y,fgmade,rad_crit
0,Team A,-23.1,3.5,0,23.363647
1,Team A,0.0,25.1,1,25.100000
2,Team A,0.5,1.0,1,1.118034
3,Team A,-5.6,5.2,0,7.641989
4,Team A,4.0,14.5,1,15.041609
...,...,...,...,...,...
555,Team B,7.8,24.7,0,25.902316
556,Team B,10.1,1.4,1,10.196568
557,Team B,-17.8,-0.1,0,17.800281
558,Team B,22.6,2.9,0,22.785302


The reason to calculate radius is to accurately differentiate the 3-pointers and the 2-points. Then, the radius is used as a criteria to account for the radius of the arc given as 23.75ft.

# Setting up criteria to classify based on zones

In [567]:
okc['zone'] = np.where((okc['y'] > 7.8) & (okc['rad_crit'] > 23.75), "NC3", "2PT") #For non-corner 3Ps and 2PTs
okc.loc[(okc['y'] <= 7.8) & (abs(okc['x']) > 22), 'zone'] = "C3" #For corner 3Ps

# Displaying rows

In [553]:
okc.head()

,team,x,y,fgmade,rad_crit,zone
0,Team A,-23.1,3.5,0,23.363647,C3
1,Team A,0.0,25.1,1,25.100000,NC3
2,Team A,0.5,1.0,1,1.118034,2PT
3,Team A,-5.6,5.2,0,7.641989,2PT
4,Team A,4.0,14.5,1,15.041609,2PT


# Using GroupBy

In [554]:
okc_gb = okc.groupby(['team','zone','fgmade'], group_keys=True)['fgmade'].size()

In [555]:
okc_gb

team    zone  fgmade
Team A  2PT   0         102
              1          68
        C3    0          12
              1           8
        NC3   0          61
              1          29
Team B  2PT   0          88
              1          75
        C3    0          16
              1           5
        NC3   0          61
              1          35
Name: fgmade, dtype: int64

# Function to determine %eFG

In [556]:
def efg(data,team,zone):
    s = data[team]
    fgpercent = ((s[zone,1])+0.5*(s[['C3','NC3']][1] + s[['C3','NC3']][3]))/(s[zone].sum())
    return round(fgpercent,3)    

# Function to determine shot distribution

In [557]:
def shot_dis(data,zone):
    team_A_percent = (data['Team A',zone].sum())/(data['Team A',zone].sum() + data['Team B',zone].sum())
    team_B_percent = 1 - team_A_percent
    return print('Shot Distribution of Team A: ' + str(round(team_A_percent, 3)) + '\n','Shot Distribution of Team B: ' + str(round(team_B_percent, 3)))

# Team A - Zone %eFG

In [558]:
efg(okc_gb, 'Team A', '2PT')

0.509

In [560]:
efg(okc_gb, 'Team A', 'NC3')

0.528

In [559]:
efg(okc_gb, 'Team A', 'C3')

1.325

# Team B - Zone %eFG

In [561]:
efg(okc_gb, 'Team B', '2PT')

0.583

In [563]:
efg(okc_gb, 'Team B', 'NC3')

0.573

In [562]:
efg(okc_gb, 'Team B', 'C3')

1.19

# Shot Distribution - Team A & Team B

In [564]:
shot_dis(okc_gb,'2PT')

Shot Distribution of Team A: 0.511
 Shot Distribution of Team B: 0.489


In [566]:
shot_dis(okc_gb,'NC3')

Shot Distribution of Team A: 0.484
 Shot Distribution of Team B: 0.516


In [565]:
shot_dis(okc_gb,'C3')

Shot Distribution of Team A: 0.488
 Shot Distribution of Team B: 0.512
